In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('q1_result.csv')

In [3]:
df.head()

,Phuong,Thang,Nam,Gia_1m
0,Phường Bến Nghé,1,2021,7.682917e+08
1,Phường Bến Nghé,1,2022,4.205095e+08
2,Phường Bến Nghé,1,2023,9.582620e+08
3,Phường Bến Nghé,1,2024,6.547570e+08
4,Phường Bến Nghé,1,2025,1.321299e+09


In [4]:
df.sort_values(by=['Phuong','Nam'], ascending=True, inplace=True)
df.reset_index(drop=True, inplace=True)

In [5]:
df['Time'] = df['Nam'].astype(str) + '-' + df['Thang'].astype(str).str.zfill(2)

In [6]:
df.drop(columns=['Nam', 'Thang'], inplace=True)

In [7]:
df.columns = ['Phuong', 'GiaTri', 'Time']
df = df[['Phuong', 'Time', 'GiaTri']]

In [8]:
df.head()

,Phuong,Time,GiaTri
0,Phường Bến Nghé,2021-01,7.682917e+08
1,Phường Bến Nghé,2021-02,8.981401e+08
2,Phường Bến Nghé,2021-03,7.041325e+08
3,Phường Bến Nghé,2021-04,6.858593e+08
4,Phường Bến Nghé,2021-05,7.010328e+08


# Chuẩn hóa

In [9]:
df.Time = pd.to_datetime(df.Time)

In [10]:
df["year"] = df["Time"].dt.year
df["month"] = df["Time"].dt.month
df["quarter"] = df["Time"].dt.quarter

In [11]:
df["month_sin"] = np.sin(2 * np.pi * df["month"]/12)
df["month_cos"] = np.cos(2 * np.pi * df["month"]/12)

In [12]:
# df = pd.get_dummies(df, columns=["Phuong"])

In [13]:
df["GiaTri_log"] = np.log1p(df["GiaTri"])

In [14]:
df = df.sort_values(["Phuong", "Time"]).reset_index(drop=True)

In [15]:
df = df[['Time', 'Phuong', 'year', 'month', 'quarter', 'month_sin', 'month_cos', 'GiaTri_log', 'GiaTri']]

In [ ]:
# giá trị của tháng trước, 2 tháng trước
for lag in [1, 2, 3]:
    df[f"GiaTri_log_lag{lag}"] = df.groupby("Phuong")["GiaTri_log"].shift(lag)

In [ ]:
# trung bình 3 tháng, 6 tháng gần nhất)
df["GiaTri_log_roll3"] = df.groupby("Phuong")["GiaTri_log"].shift(1).rolling(window=3).mean()
df["GiaTri_log_roll6"] = df.groupby("Phuong")["GiaTri_log"].shift(1).rolling(window=6).mean()
df = df.dropna().reset_index(drop=True)

In [18]:
df.head(10)

,Time,Phuong,year,month,quarter,month_sin,month_cos,GiaTri_log,GiaTri,GiaTri_log_lag1,GiaTri_log_lag2,GiaTri_log_lag3,GiaTri_log_roll3,GiaTri_log_roll6
0,2021-07-01,Phường Bến Nghé,2021,7,3,-5.000000e-01,-8.660254e-01,20.454367,7.642206e+08,20.478504,20.368065,20.346183,20.397584,20.440124
1,2021-08-01,Phường Bến Nghé,2021,8,3,-8.660254e-01,-5.000000e-01,20.873333,1.161913e+09,20.454367,20.478504,20.368065,20.433646,20.439239
2,2021-09-01,Phường Bến Nghé,2021,9,3,-1.000000e+00,-1.836970e-16,20.428921,7.450199e+08,20.873333,20.454367,20.478504,20.602068,20.482155
3,2021-10-01,Phường Bến Nghé,2021,10,4,-8.660254e-01,5.000000e-01,20.378369,7.082932e+08,20.428921,20.873333,20.454367,20.585541,20.491562
4,2021-11-01,Phường Bến Nghé,2021,11,4,-5.000000e-01,8.660254e-01,20.588378,8.738137e+08,20.378369,20.428921,20.873333,20.560208,20.496927
5,2021-12-01,Phường Bến Nghé,2021,12,4,-2.449294e-16,1.000000e+00,20.451922,7.623546e+08,20.588378,20.378369,20.428921,20.465223,20.533645
6,2022-01-01,Phường Bến Nghé,2022,1,1,5.000000e-01,8.660254e-01,19.856978,4.205095e+08,20.451922,20.588378,20.378369,20.472890,20.529215
7,2022-02-01,Phường Bến Nghé,2022,2,1,8.660254e-01,5.000000e-01,20.743160,1.020093e+09,19.856978,20.451922,20.588378,20.299093,20.429650
8,2022-03-01,Phường Bến Nghé,2022,3,1,1.000000e+00,6.123234e-17,20.244688,6.196642e+08,20.743160,19.856978,20.451922,20.350687,20.407955
9,2022-04-01,Phường Bến Nghé,2022,4,2,8.660254e-01,-5.000000e-01,20.480911,7.847780e+08,20.244688,20.743160,19.856978,20.281608,20.377249


In [20]:
df.to_csv('data.csv', index=False)

# Model

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, r2_score

In [22]:
features = [
    "year", "month", "quarter", "month_sin", "month_cos",
    "GiaTri_log_lag1", "GiaTri_log_lag2", "GiaTri_log_lag3",
    "GiaTri_log_roll3", "GiaTri_log_roll6"
]
target = "GiaTri_log"

# Train-test split theo thời gian
split_point = int(len(df) * 0.8)
train_df = df.iloc[:split_point]
test_df = df.iloc[split_point:]


X_train, y_train = train_df[features], train_df[target]
X_test, y_test = test_df[features], test_df[target]

print("Train shape:", X_train.shape, y_train.shape)
print("Test shape:", X_test.shape, y_test.shape)

Train shape: (386, 10) (386,)
Test shape: (97, 10) (97,)


In [24]:
model = ElasticNet(alpha=0.01, l1_ratio=1.0, random_state=42)

# Train
model.fit(X_train, y_train)

# Dự đoán
y_pred = model.predict(X_test)

# Đánh giá
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("ElasticNet RMSE:", rmse)
print("ElasticNet R2:", r2)


ElasticNet RMSE: 0.30151217465657004
ElasticNet R2: -0.07333781550881757


# Biểu đồ so sánh

In [35]:
import plotly.graph_objects as go

# Nếu y_test, y_pred đang là dạng log, nhớ inverse_transform trước
y_true = y_test.values.flatten()
y_pred_vals = y_pred.flatten()

fig = go.Figure()

# Giá trị thực tế
fig.add_trace(go.Scatter(
    y=y_true,
    # x=X_test['year'].astype(str) + '-' + X_test['month'].astype(str).str.zfill(2),
    mode="lines+markers",
    name="Thực tế"
))

# Giá trị dự đoán
fig.add_trace(go.Scatter(
    y=y_pred_vals,
    # x=X_test['year'].astype(str) + '-' + X_test['month'].astype(str).str.zfill(2),
    mode="lines+markers",
    name="Dự đoán"
))

fig.update_layout(
    title="So sánh giá trị thực tế và dự đoán (ElasticNet)",
    xaxis_title="Thời gian",
    yaxis_title="Giá trị",
    template="plotly_white"
)

fig.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go

# --- So sánh thực tế và dự đoán trên tập test ---
fig = go.Figure()

# Đường thực tế
fig.add_trace(go.Scatter(
    y=y_test,
    mode='lines+markers',
    name='Thực tế'
))

# --- Dự đoán 1 điểm tương lai ---
X_last = X_test.iloc[-1].values.reshape(1, -1)
future_pred = model.predict(X_last)[0]

# Ghép thêm điểm mới vào chuỗi dự đoán
y_pred_extended = np.append(y_pred, future_pred)

# Đường dự đoán (đã nối điểm mới)
fig.add_trace(go.Scatter(
    y=y_pred_extended,
    mode='lines+markers',
    name='Dự đoán + Tương lai'
))

# Layout
fig.update_layout(
    title="So sánh giá trị thực tế, dự đoán và điểm tương lai (ElasticNet)",
    xaxis_title="Thời gian (index)",
    yaxis_title="Giá trị chuẩn hóa",
    template="plotly_white"
)

fig.show()


/home/bangbui/miniforge3/envs/work/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning:

X does not have valid feature names, but ElasticNet was fitted with feature names



# Save model

In [30]:
import joblib

# Lưu model
joblib.dump(model, "model/elasticnet_model.pkl")

# Khi cần load lại
loaded_model = joblib.load("model/elasticnet_model.pkl")

# Dùng lại để dự đoán
y_loaded_pred = loaded_model.predict(X_test)